# Рассчет эфемерид микролинзирования

In [93]:
from math import *
import numpy as np
from datetime import datetime, date
from astropy.time import Time
from astroquery.jplhorizons import Horizons


### Функция для вычисления положения звезды в экваториальных декартовых координатах с учетом параллактического смещения и собственного движения 
 $\alpha, \delta$ - экваториальные координаты в опорную эпоху $t_0$ ,
 $\mu_{\delta}, \mu_{\alpha} = \mu_{\alpha^*}/\cos{\delta}$ - компоненты собственного движения,
 $dist = 1/\varpi$ - расстояние до объекта,
 $\boldsymbol{E}(t)$ - это геоцентрическое положение Солнца в экваториальных декартовых координатах на момент времени $t$
\begin{equation*}
\boldsymbol{x}(t) = \begin{pmatrix} \cos(\delta + \mu_{\delta}(t-t_0)) \cos(\alpha + \mu_{\alpha}(t-t_0)) \\ \cos(\delta + \mu_{\delta}(t-t_0)) \sin(\alpha + \mu_{\alpha}(t-t_0)) \\ \sin(\delta + \mu_{\delta}(t-t_0)) \end{pmatrix} \cdot dist + \boldsymbol{E}(t)
\end{equation*}


In [94]:
def coord(ra, dec, t0, t, plx, pmra, pmdec):
    #all parameters in degrees
    plx = plx*3600
    pmra = pmra/cos(radians(dec))
    ra1, dec1 = map(radians, [ra +  pmra*(t - t0), dec + pmdec*(t - t0)])
    x0 = np.array([cos(dec1)*cos(ra1), cos(dec1)*sin(ra1), sin(dec1)])*206265/plx
    JD = Time(t, format = 'jyear').jd
    obj = Horizons(id='sun', id_type='majorbody', location='399', epochs=JD)
    ev= obj.vectors(refplane='earth')
    E = np.array([ ev['x'][0], ev['y'][0], ev['z'][0]])
    
    return x0 + E

### Входные данные источника и линзы для дальнейшего рассчета эфемерид микролинзирования

In [95]:
t0 = 2016.0

#Для рассчета эфемерид микролинзирования необходимо ввести координаты на момент времени t0, собственные движения,
# параллаксы, звездные величины для линзы и источника, массу линзы, время (в примерах ниже время наибольшего сближения)

events = [
{'ra_s': 280.40180044, 'dec_s': 0.90750181, 'plx_s': 0.61488754, 'pmra_s': -2.0213466, 'pmdec_s': -6.317931,
'ra_l': 280.40170318, 'dec_l': 0.91170624, 'plx_l': 33.253544, 'pmra_l': 46.01097, 'pmdec_l': -1981.2155,
't': 2023.6639973853855, 'mag_l': 11.555, 'mag_s': 20.4161, 'M': 0.4187125 },

{'ra_s': 342.20166306, 'dec_s': 45.33585263, 'plx_s': 0.59739923, 'pmra_s': -5.3419695, 'pmdec_s': -5.1250076,
'ra_l': 342.20107927, 'dec_l': 45.33670251, 'plx_l': 32.53825, 'pmra_l': 176.0452, 'pmdec_l': -380.09622,
't': 2024.1175280033751, 'mag_l': 13.596, 'mag_s': 17.0734, 'M': 0.22968246 },
    
{'ra_s': 6.21763794, 'dec_s': 68.57991699, 'plx_s': 1.0019236, 'pmra_s': -0.7912279, 'pmdec_s': 2.801217,
'ra_l': 6.21486490, 'dec_l': 68.57978306, 'plx_l': 27.757534, 'pmra_l': 411.58395, 'pmdec_l': 23.018646,
't': 2024.7601512315396, 'mag_l': 17.4672, 'mag_s': 20.8068, 'M': 0.65 }    
]

i = 2  # номер события в списке events

# параметры источника на t0
ra_s, dec_s = events[i]['ra_s'], events[i]['dec_s'] #в градусах
plx_s = events[i]['plx_s'] #в милисекундах дуги
pmRA_s, pmDEC_s = events[i]['pmra_s'], events[i]['pmdec_s'] #в милисекундах дуги в год

#параметры линзы на t0
ra_l, dec_l = events[i]['ra_l'], events[i]['dec_l']
plx_l = events[i]['plx_l']
pmRA_l, pmDEC_l = events[i]['pmra_l'], events[i]['pmdec_l']

t = events[i]['t'] # время в долях года
mag_l = events[i]['mag_l'] # звездная величина линзы
mag_s = events[i]['mag_s'] # звездная величина источника
f_ls = 10**(0.4*(mag_s - mag_l)) # отношение потоков линзы и источника
M = events[i]['M'] # масса линзы




### Угловое расстояние между линзой и источником
\begin{equation}
    d(t) = 2 \cdot \arcsin \left( \left| \frac{\boldsymbol{x_{l}}(t)}{|\boldsymbol{x_{l}}(t)|} - \frac{\boldsymbol{x_{s}}(t)}{|\boldsymbol{x_{s}}(t)|} \right| \cdot 0.5 \right)
\end{equation}

In [96]:
def angdist(t):
    #return distance between lens and source in degrees
    s = coord(ra_s, dec_s, t0, t, plx_s/3.6e6, pmRA_s/3.6e6, pmDEC_s/3.6e6)
    l = coord(ra_l, dec_l, t0, t, plx_l/3.6e6, pmRA_l/3.6e6, pmDEC_l/3.6e6)
    d = 2*asin(0.5*np.linalg.norm(l/np.linalg.norm(l) - s/np.linalg.norm(s)))
    return degrees(d)

### Создаем класс микролинзирования с методами неразрешеннего и частично-разрешенного микролинзирования. Рассматриваются случаи с "темной" и "яркой" линзой



In [97]:
class Microlensing:
    '''M - масса линзы в массах солнца
    plx_x, plx_s - в mas, координатные векторы - в градусах
    t - в юлианских днях'''
    def __init__(self, angdist, t, M, plx_l, plx_s):
        self.d = angdist(t)*3.6e6  #dist in mas
        
        
        #const = sqrt((2*1.32712442076*(10**20)/299792458**2)*2*206264.8*10**3/149597870700)
        const = 2.854
        
        # радиус конуса Эйнштейна в милисекундах дуги
        self.theta_e = const*sqrt(M*(plx_l - plx_s))
        #прицельный параметр
        self.u = self.d/self.theta_e
        
        #Увеличения потоков для I1, I2:
        self.A1 = 0.5 + (self.u**2 + 2)/(2*self.u*sqrt(self.u**2 + 4))
        self.A2 = self.A1 - 1
        #общее увеличение системы для темной линзы:
        self.A = self.A1 + self.A2
        
        # расстояние L − I1 в mas
        self.theta_1 = self.theta_e*(sqrt(self.u**2 + 4) + self.u)/2
        # расстояние L − I2 / S − I1 в mas
        self.theta_2 = self.theta_1 - self.theta_e*self.u
        
        
    def unresolved_microlensing(self, f_ls):
        '''Случай линзирования, когда изображения и линза неразрешены.
        Принимает на вход отношение потоков источника и линзы.
        '''
        #общее увеличение потока системы в случае дополнительного источника света
        A_lum = (self.A + f_ls)/(f_ls + 1)
        dmag = 2.5*log10(A_lum)
        
        #расстояние от линзы до центроида света линзы и источника при микролинзировании в mas
        theta_mic = (self.A1*self.theta_1 - self.A2*self.theta_2)/(self.A1 + self.A2 + f_ls)
        #расстояние от линзы до центроида света линзы и источника без микролинзирования в mas
        theta_ls = self.theta_e*self.u/(1 + f_ls)
        # смещение центроида света
        shift_lum = theta_mic - theta_ls
        
        #смещение центроида света изображений для темной линзы относительно источника:
        shift = self.u*self.theta_e/(self.u**2 + 2)
        
        return {'A_lum': A_lum, 'dm': dmag, 'theta_mic': theta_mic, 'theta_ls': theta_ls, 'centroid_shift': shift, 'centroid_shift_lum' : shift_lum}
    
    
    def partial_microlensing(self, f_ls):
        '''Случай микролинзирования с частичным разрешением, когда 1 изображение и линза неразрешены.
        Принимает на вход потоки источника и линзы.
        Возвращает увеличения потока системы 2 изображения и с линзой,
        угловое расстояние центроида 2 изображения и линзы, отсчитываемое от линзы'''
        A_2l = (self.A2 + f_ls)/f_ls

        theta_2l = self.A2*self.theta_2/(self.A2 + f_ls)
        
        return {'A_2l': A_2l, 'theta_2l': theta_2l}
        

### Подставляем циферки и смотрим, какие параметры микролинзирования получаются

In [98]:
event1 = Microlensing(angdist, t, M, plx_l, plx_s)
print(f"theta_e: {event1.theta_e} mas \n u: {event1.u}\n A1: {event1.A1}\n A2: {event1.A2}\n A: {event1.A} \n theta_i1s: {event1.theta_2} mas")
print(f"unresolved_microlensing: {event1.unresolved_microlensing(f_ls)}\n partial_microlensing: {event1.partial_microlensing(f_ls)}")

theta_e: 11.901948830446893 mas 
 u: 23.80840708732511
 A1: 1.0000030904618886
 A2: 3.0904618886218316e-06
 A: 1.0000061809237772 
 theta_i1s: 0.49902647996628957 mas
unresolved_microlensing: {'A_lum': 1.0000002726590673, 'dm': 2.960357805909407e-07, 'theta_mic': 12.522191601959713, 'theta_ls': 12.500142831795584, 'centroid_shift': 0.49814766776130553, 'centroid_shift_lum': 0.02204877016412965}
 partial_microlensing: {'A_2l': 1.0000001426209721, 'theta_2l': 7.117163152748193e-08}
